In [1]:
# libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import random

personal_directory = '/home/jan/Uni/DS-Project/'

### create data to show in dashboard


In [2]:
region_namen = pd.read_csv(personal_directory + 'modules/dashboard/test/apps/assets/region_name_trans.csv')

In [3]:
# Gemeinden
# Load the GeoJSON file
with open(personal_directory + 'modules/dashboard/test/apps/assets/BB_gemeinde_final.json') as f:
    geojson = f.read()

# Convert the GeoJSON string to a Python dictionary
geojson_dict = json.loads(geojson)

gemeinde = gpd.GeoDataFrame.from_features(geojson_dict['features'])
gemeinde = gemeinde[['NAME_3', 'suitable_area', 'terrain_score', 'irradiation_score', 'distance_score', 'overall_score']]

# join df with extra regions
region_namen['NAME_3'] = region_namen['NAME_3'].astype('object')
gemeinde_df = gemeinde.merge(region_namen, on='NAME_3', how='left')

In [68]:
print(gemeinde_df['NAME_2'].isna().sum() / len(gemeinde_df.NAME_2))
print(gemeinde_df['NAME_2'].isna().sum())

0.18052194953335637
2089


In [69]:
# Kreise
# Load the GeoJSON file
with open(personal_directory + 'modules/dashboard/test/apps/assets/BB_kreis_final.json') as f:
    geojson = f.read()

# Convert the GeoJSON string to a Python dictionary
geojson_dict = json.loads(geojson)

kreise = gpd.GeoDataFrame.from_features(geojson_dict['features'])
kreise = kreise[['NAME_2', 'terrain_score', 'irradiation_score', 'distance_score', 'overall_score']]

# join df with extra regions
region_namen['NAME_2'] = region_namen['NAME_2'].astype('object')
kreise_df = kreise.merge(region_namen[['NAME_1', 'NAME_2']], on='NAME_2', how='left')

In [70]:
kreise_df['NAME_1'].isna().sum()

0

In [71]:
# Zusatzinfos
# Load the GeoJSON file
with open(personal_directory + 'data/Maps/georef-germany-gemeinde-millesime.geojson') as f:
    geojson = f.read()

# Convert the GeoJSON string to a Python dictionary
geojson_dict = json.loads(geojson)
zusatz = gpd.GeoDataFrame.from_features(geojson_dict['features'])
zusatz = zusatz[['lan_name', 'krs_name', 'gem_name_short']]

zusatz = zusatz.astype(str)  # convert each element to string
zusatz = zusatz.applymap(lambda x: x.strip("['']"))  # remove brackets from all columns
zusatz['krs_name'] = zusatz['krs_name'].apply(lambda x: x.split(' ', 1)[1])  # split the second column and keep only the second part
zusatz = zusatz[['lan_name', 'krs_name', 'gem_name_short']]  # select desired columns

# rename columns
# create a dictionary of the new column names
new_columns = {'lan_name': 'NAME_1', 'krs_name': 'NAME_2', 'gem_name_short': 'NAME_3'}

# rename the columns using the rename() method
zusatz = zusatz.rename(columns=new_columns)

In [72]:
zusatz.head()

,NAME_1,NAME_2,NAME_3
0,Saarland,Saarlouis,Rehlingen-Siersburg
1,Saarland,Saarpfalz-Kreis,Mandelbachtal
2,Brandenburg,Dahme-Spreewald,Neu Zauche
3,Brandenburg,Dahme-Spreewald,Steinreich
4,Brandenburg,Elbe-Elster,Schönborn


In [73]:
gemeinde_test = gemeinde.merge(zusatz, on='NAME_3', how='left')

In [74]:
print(gemeinde_test['NAME_2'].isna().sum() / len(gemeinde_test.NAME_2))
print(gemeinde_test['NAME_2'].isna().sum())

0.37282085339531795
4491


In [75]:
kreise_df.to_csv('assets/kreise_df.csv')
gemeinde_df.to_csv('assets/gemeinde_df.csv')